In [151]:
import os
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Activation, merging, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import Image
import re

In [159]:
#フォルダ名をクラス名にする
path = "./sns/train"
folders = os.listdir(path)
classes = [f for f in folders if os.path.isdir(os.path.join(path, f))]
n_classes = len(classes)
print("n_classes", n_classes)
print(classes)

n_classes 5
['facebook', 'line', 'tiktok', 'twitter', 'twitter_t']


In [180]:
#画像の読み込みとフォルダ番号の答え
X =[]
y=[]
for label, class_name in enumerate(classes):
    files= glob("./sns/train/" + class_name+"/*.*")
    for file in files:
        img=cv2.imread(file, 0)
        img = cv2.resize(img, dsize=(224,224))
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        X.append(img)
        y.append(label)
print(len(X))
print(len(y))
print(X[0].shape)

195
195
<class 'list'>
(224, 224, 3)


In [181]:
#ndarray化
X_ = np.array(X)
X_ = X_.astype("float32")
X_train = X_/255.0
y_ = np.array(y)
y_train = np_utils.to_categorical(y_,n_classes)

In [182]:
answer = {"0":"facebook", "1":"line", "2":"tiktk","3":"twitter","4":"twitter_t"}
answer

{'0': 'facebook', '1': 'line', '2': 'tiktk', '3': 'twitter', '4': 'twitter_t'}

In [183]:
X_test =[]
y_test=[]
for label, class_name in enumerate(classes):
    files= glob("./sns/validation/" + class_name+"/*.*")
    for file in files:
        img = cv2.imread(file,0)
        img = cv2.resize(img, dsize=(224,224))
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        X_test.append(img)
        y_test.append(label)
X_test = np.array(X_test)
X_test = X_test.astype("float32")
X_test = X_test/255
y_test = np.array(y_test)
y_test = np_utils.to_categorical(y_test,n_classes)
print(len(X_test))
print(len(y_test))

147
147


In [187]:
X_test[0].shape

(224, 224, 3)

In [188]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(195, 224, 224, 3)
(195, 5)
(147, 224, 224, 3)
(147, 5)


In [145]:
#vgg16
input_tensor = Input(shape=(224,224,3))
base_model = VGG16(weights="imagenet", input_tensor=input_tensor, include_top=False)

In [146]:
#後付
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes, activation="softmax"))

In [147]:
#"結合
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [148]:
#学習させない層
for layer in model.layers[:15]:
    layer.trainable = False
print('# layers = ', len(model.layers))

# layers =  20


In [149]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [186]:
model.fit(X_train, y_train, epochs=20, batch_size=16)

Epoch 1/20


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [112]:
score = model.evaluate(X_test, y_test, batch_size=16)

10/10 [==============================] - 1s 37ms/step - loss: 1.0661 - accuracy: 0.9048


In [47]:
import pickle
#クラス名の保存
pickle.dump(classes, open("classes.sav","wb"))
#モデルの保存
model.save("./cnn.h5")

In [49]:
from keras.models import load_model
import pickle
import cv2

In [66]:
images = [image for image in glob("./sns/testyou/*.*")]
images[3]

'./sns/testyou\\f3.jpg'

In [196]:
numbers = [re.search(r"\\(.*)",number).group(1) for number in images]
numbers[33]

't32.jpg'

In [ ]:
results = []
numbers = []
for image in images:
    img = cv2.imread(image)
    img = cv2.resize(img, dsize=(224,224))
    img = img.astype('float32')
    img /= 255.0
    img = img[None, ...]
    results.append(model.predict(img))
    numbers.append(re.search(r"u\\(.*.jpg)",image).group(1))
len(results)

In [63]:
#1枚テスト
img = cv2.imread("./sns/testyou/tk50.jpg")
img = cv2.resize(img, dsize=(224,224))
img = img.astype('float32')
img /= 255.0
img = img[None, ...]
result = model.predict(img)

1/1 [==============================] - 0s 31ms/step


In [64]:
np.set_printoptions(precision=3, suppress=True)
result * 100

array([[  0.,   0., 100.,   0.,   0.]], dtype=float32)

In [114]:
#予測収納
answers = []
for result in results:
    np.set_printoptions(precision=3, suppress=True)
    a = result * 100
    answers.append(a)
len(answers)

59

In [197]:
for a,name in zip(answers, numbers):
    k = a.argmax()
    # print(name,k)
    print(name, answer[str(k)])

f1.jpg facebook
f10.jpg facebook
f2.jpg facebook
f3.jpg facebook
f4.jpg facebook
f5.jpg facebook
f6.jpg facebook
f7.jpg facebook
f8.jpg facebook
f9.jpg facebook
l.jpg line
l11.jpg line
l13.jpg line
l14.jpg line
l15.jpg line
l16.jpg line
l17.jpg line
l19.jpg line
l20.jpg line
l21.jpg line
l22.jpg line
l23.jpg line
l24.jpg line
l25.jpg line
l26.jpg line
l27.jpg line
l28.jpg line
l29.jpg line
l30.jpg line
lk18.jpg line
t.jpg twitter
t0.jpg twitter
t31.jpg facebook
t32.jpg facebook
t33.jpg facebook
t34.jpg facebook
t35.jpg twitter
t36.jpg twitter
t37.jpg twitter
t38.jpg twitter
t39.jpg twitter
t40.jpg twitter
t41.jpg twitter
t42.jpg twitter
t43.jpg facebook
t44.jpg facebook
t45.jpg twitter
t46.jpg twitter
t57.jpg twitter
t58.jpg twitter
tk47.jpg tiktk
tk48.jpg tiktk
tk49.jpg tiktk
tk50.jpg tiktk
tk52.jpg tiktk
tk53.jpg tiktk
tk54.jpg tiktk
tk55.jpg tiktk
tk56.jpg tiktk


In [65]:
a = result.argmax()
answer[str(a)]

'tiktk'